### This is trying to make some sense of Francesco raman variables 

##### Using Python version 3.11.9 and packages below:

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import integrate

In [10]:

# Define the neural network 
class RamanNet(nn.Module):
    def __init__(self, input_size):
        super(RamanNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Custom dataset 
class RamanDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def area_normalize_spectrum(spectrum):
    """Normalize a single spectrum by its area."""
    area = integrate.simps(spectrum)
    return spectrum / area if area != 0 else spectrum

def load_and_normalize_data(file_list):
    data = []
    concentrations = []
    for file in file_list:
        df = pd.read_csv(file)
        
        # Filter data to 300-1800 cm^-1 range
        mask = (df['Raman shift'] >= 300) & (df['Raman shift'] <= 1800)
        filtered_df = df[mask]
        
        normalized_spectrum = area_normalize_spectrum(filtered_df['intensity'].values)
        data.append(normalized_spectrum)
        
        # Extract concentration from filename
        conc = float(file.split('_')[0].replace(',','.'))
        concentrations.append(conc)
    
    return np.array(data), np.array(concentrations)

# Load and normalize datasets
concentrations = [0, 20, 33.3, 50, 66.6, 75, 100]
all_data = []
all_concentrations = []


def list_csv_files(directory):
    """List all CSV files in a given directory."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

X = np.array(all_data)
y = np.array(all_concentrations)

# Directory containing CSV files
directory = 'C:/Users/baubl/OneDrive/Dokumentai/GitHub/raman-pigments-model/DATA/CSV files'

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = RamanDataset(X_train, y_train)
test_dataset = RamanDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = RamanNet(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 200
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    model.eval()
    epoch_test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            epoch_test_loss += loss.item()
    
    train_losses.append(epoch_train_loss / len(train_loader))
    test_losses.append(epoch_test_loss / len(test_loader))
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(torch.FloatTensor(X_test)).numpy().flatten()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()

# Plot predictions vs true values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', label='Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect predictions')
plt.xlabel('True concentration')
plt.ylabel('Predicted concentration')
plt.legend()
plt.title('Green Pigment Concentration Prediction (300-1800 cm^-1, PyTorch Model)')
plt.show()

# Function to predict on new data
def predict_new_data(new_spectrum, raman_shift, model):
    # Filter new spectrum to 300-1800 cm^-1 range
    mask = (raman_shift >= 300) & (raman_shift <= 1800)
    filtered_spectrum = new_spectrum[mask]
    
    normalized_new_spectrum = area_normalize_spectrum(filtered_spectrum)
    normalized_new_spectrum = torch.FloatTensor(normalized_new_spectrum).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        prediction = model(normalized_new_spectrum).item()
    
    return prediction



ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
def read_dpt_file(file_path):
    # Extract concentration from filename
    filename = os.path.basename(file_path)
    concentration = float(filename.split('.')[0])
    # Read the file into a DataFrame
    data = pd.read_csv(file_path, delimiter=',', header=None, names=['wavelength', 'intensity'])
    data['concentration'] = concentration
    return data

In [23]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
from scipy import integrate

# Define the neural network
class RamanNet(nn.Module):
    def __init__(self, input_size):
        super(RamanNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Custom dataset
class RamanDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def area_normalize_spectrum(spectrum):
    """Normalize a single spectrum by its area."""
    area = integrate.simps(spectrum)
    return spectrum / area if area != 0 else spectrum

def load_and_normalize_data(file_list):
    data = []
    concentrations = []
    for file in file_list:
        df = pd.read_csv(file)

        # Rename columns to 'Raman shift' and 'intensity'
        df.columns = ['Raman shift', 'intensity']
        
        # Print columns for debugging
        print(f"Columns in {file}: {df.columns.tolist()}")

        if 'Raman shift' not in df.columns or 'intensity' not in df.columns:
            print(f"File {file} does not contain the required columns.")
            continue
        
        # Filter data to specific ranges based on the provided image
        mask = ((df['Raman shift'] >= 300) & (df['Raman shift'] <= 450)) | \
               ((df['Raman shift'] >= 500) & (df['Raman shift'] <= 700)) | \
               ((df['Raman shift'] >= 1200) & (df['Raman shift'] <= 1600))
        filtered_df = df[mask]
        
        normalized_spectrum = area_normalize_spectrum(filtered_df['intensity'].values)
        print(normalized_spectrum)
        data.append(normalized_spectrum)
        
        # Extract concentration from filename
        concentration_part = os.path.basename(file).split('_')[0]
        numeric_string = concentration_part.replace('.csv', '')
        conc = float(numeric_string)
        concentrations.append(conc)
    
    return np.array(data), np.array(concentrations)

def list_csv_files(directory):
    """List all CSV files in a given directory."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

# Directory containing CSV files
directory = 'C:/Users/baubl/OneDrive/Dokumentai/GitHub/raman-pigments-model/DATA/CSV files'  # Adjust the path to the directory

# List all CSV files in the directory
file_list = list_csv_files(directory)

# Load and normalize all datasets
X, y = load_and_normalize_data(file_list)

# Create datasets
dataset = RamanDataset(X, y)

# Bootstrap parameters
num_bootstrap_samples = 10
num_epochs = 200
bootstrap_results = []

for i in range(num_bootstrap_samples):
    print(f"Bootstrap Sample {i+1}")
    
    # Create a bootstrap sample
    indices = np.arange(len(dataset))
    bootstrap_indices = resample(indices, replace=True, n_samples=len(indices))
    oob_indices = np.setdiff1d(indices, bootstrap_indices)
    
    train_dataset = Subset(dataset, bootstrap_indices)
    test_dataset = Subset(dataset, oob_indices)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Initialize the model, loss function, and optimizer
    input_size = X.shape[1]
    model = RamanNet(input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()

        model.eval()
        epoch_test_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.unsqueeze(1))
                epoch_test_loss += loss.item()

        train_losses.append(epoch_train_loss / len(train_loader))
        test_losses.append(epoch_test_loss / len(test_loader))

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_test_actual = []
        y_test_predicted = []
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            y_test_actual.extend(batch_y.numpy())
            y_test_predicted.extend(outputs.numpy().flatten())

    mse = mean_squared_error(y_test_actual, y_test_predicted)
    r2 = r2_score(y_test_actual, y_test_predicted)
    print(f"Bootstrap Sample {i+1} - MSE: {mse:.4f}, R2 Score: {r2:.4f}")
    bootstrap_results.append((mse, r2))

# Aggregate bootstrap results
avg_mse = np.mean([result[0] for result in bootstrap_results])
avg_r2 = np.mean([result[1] for result in bootstrap_results])
print(f"Average MSE: {avg_mse:.4f}")
print(f"Average R2 Score: {avg_r2:.4f}")

# Plot the training and testing loss curves for the last bootstrap sample
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Learning Curve for the Last Bootstrap Sample')
plt.legend()
plt.show()

# Function to predict on new data
def predict_new_data(new_spectrum, raman_shift, model):
    # Filter new spectrum to specific ranges based on the provided image
    mask = ((raman_shift >= 300) & (raman_shift <= 450)) | \
           ((raman_shift >= 500) & (raman_shift <= 700)) | \
           ((raman_shift >= 1200) & (raman_shift <= 1600))
    filtered_spectrum = new_spectrum[mask]
    
    normalized_new_spectrum = area_normalize_spectrum(filtered_spectrum)
    normalized_new_spectrum = torch.FloatTensor(normalized_new_spectrum).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        prediction = model(normalized_new_spectrum).item()
    
    return prediction

df = pd.read_csv('C:/Users/baubl/OneDrive/Dokumentai/GitHub/raman-pigments-model/DATA/CSV')
df
# Example of predicting on new data
def load_new_spectrum(file_path):
    df = pd.read_csv(file_path)
    return df['Raman shift'].values, df['intensity'].values

# Example usage:
# raman_shift, new_spectrum = load_new_spectrum('path/to/new_sample.csv')
# prediction = predict_new_data(new_spectrum, raman_shift, model)
# print(f"Predicted concentration: {prediction:.2f}")

       300  37.63506841
0      302    58.079583
1      304    27.920075
2      306    78.469875
3      308    90.981969
4      310   115.169441
...    ...          ...
1445  3192    87.990527
1446  3194   182.841618
1447  3196   293.827253
1448  3198   202.655330
1449  3200   124.913691

[1450 rows x 2 columns]
Columns in C:/Users/baubl/OneDrive/Dokumentai/GitHub/raman-pigments-model/DATA/CSV files\0.0.csv: ['Raman shift', 'intensity']
[1.88033710e-04 9.03917520e-05 2.54047652e-04 2.94555784e-04
 3.72863168e-04 4.85171471e-04 5.56783314e-04 5.92688687e-04
 6.07049284e-04 6.51861590e-04 7.69745982e-04 7.92022128e-04
 7.40610819e-04 7.35812244e-04 8.11913435e-04 8.68844725e-04
 7.93515103e-04 7.68480539e-04 7.25388105e-04 7.51844577e-04
 7.01235799e-04 7.71769136e-04 7.64536199e-04 8.95807715e-04
 9.57320084e-04 1.01918193e-03 1.03200811e-03 1.03458460e-03
 1.05559874e-03 1.05395137e-03 1.18979786e-03 1.20466078e-03
 1.13549262e-03 9.43959615e-04 9.19742192e-04 9.01651446e-04
 9.87557287

KeyboardInterrupt: 